In [6]:
import pandas as pd
import scanpy as sc

In [7]:
import os
EC_path = '../multi_region_single_cell/EC/minor_cell_type/'
adata = sc.read_h5ad(f'{EC_path}Ast.h5ad')

In [8]:
cell_types = []
for i in os.listdir(EC_path):
    if '.h5ad' in i:
        cell_types.append(i)
labels  = ['clinical_AD_Label','Pathological_AD_Label'] 
cell_types

['Per.h5ad',
 'Opc.h5ad',
 'Oli.h5ad',
 'Ast.h5ad',
 'Mic.h5ad',
 'SMC.h5ad',
 'End.h5ad',
 'Inh.h5ad',
 'Fib.h5ad',
 'CAM.h5ad',
 'CPEC.h5ad',
 'T.h5ad',
 'Exc.h5ad',
 'Epd.h5ad']

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [10]:
all_genes = adata.var.varnames.tolist()

In [11]:
import numpy as np

def return_ensemble_id(x,df_ensemble):
    try:
        return df_ensemble[df_ensemble['1']==x]['0'].values[0]
    except: 
        return np.nan

def return_clinical_label(x):
    if x['clinical_diagnosis'] == 'AD':
        return 1.0
    elif x['clinical_diagnosis'] == 'NCI':
        return 0.0
    else:
        return 2.0


def return_pthological_label(x):
    if x['pathological_diagnosis'] == 'AD':
        return 1.0
    elif x['pathological_diagnosis'] == 'No AD':
        return 0.0
    else:
        return 2.0

def return_clinical_pthological_label(x):
    if x['clinical_pathological_AD'] == 'AD_with_Plaques':
        return 1.0
    elif x['clinical_pathological_AD'] == 'NCI_with_No_Plaques':
        return 0.0
    else:
        return 2.0

# Evaluation


clinical_data = pd.read_csv('../clinical_single_cell.csv')

df_ensemble = pd.read_csv('../df_ensemble.csv')

cell_types = []
for i in os.listdir(EC_path):
    if '.h5ad' in i:
        cell_types.append(i)

gene_excel = pd.ExcelFile('../202141022_Genelist_m6aproject_BED.xlsx')
m6a = pd.read_excel(gene_excel, 'm6A')
m6a = m6a.iloc[2:,]
m6a_genes = m6a['m6A related genes/proteins'].tolist()

mito = pd.read_excel(gene_excel, 'Mitochondria')
mito  = mito.iloc[2:,]
mito_genes = mito['Mitochondria genes'].tolist()


mitophagy = pd.read_excel(gene_excel, 'Mitophagy')
mitophagy = mitophagy.iloc[2:,]
mitophagy_genes = mitophagy['Autophagy_Mitophagy_lysosome'].tolist()

AD = pd.read_excel(gene_excel, 'AD')
AD   = AD.iloc[2:,]
AD_genes = AD["Alzheimer's disease - risk factors"].tolist()


mito = pd.read_excel(gene_excel, 'Mitochondria')
mito  = mito.iloc[2:,]
mito_genes = mito['Mitochondria genes'].tolist()
mito_genes[0:5]

lysosome = pd.read_excel(gene_excel, 'Lysosome')
lysosome  = lysosome.iloc[2:,]
lysosome_genes = lysosome['Lysosomes'].tolist()

genes_set  =  [all_genes,m6a_genes,mito_genes,mitophagy_genes,AD_genes,mito_genes,lysosome_genes]
genes_set_name  =  ['all_genes','m6a_genes','mito_genes','mitophagy_genes','AD_genes','mito_genes','lysosome_genes']


        
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            outputs = model(inputs).squeeze(1)
            preds = torch.sigmoid(outputs) > 0.5  # Convert logits to binary predictions
            correct += (preds == labels.squeeze(1)).sum().item()
            total += labels.size(0)
            Accuracy = 100 * correct / total
    return Accuracy

def train_model(model, train_loader,test_loader, criterion, optimizer, epochs=10):
    model.train()
    best_accuracy = 0
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(inputs).squeeze(1)  # Remove extra dimension
            loss = criterion(outputs, labels.squeeze(1).float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        test_accuracy  = evaluate_model(model, test_loader)
        train_accuracy = evaluate_model(model, train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}, TrainAccuracy: {train_accuracy:.2f}, Test_Accuracy: {test_accuracy:.2f}")
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
    
    return best_accuracy




class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

accuracy_dct = dict()

for i in cell_types:
    print(i)
    adata = sc.read_h5ad(f'{EC_path}{i}')
    temp_data = pd.merge(adata.obs.reset_index(), clinical_data, on =  'projid', how='inner').drop_duplicates()
    temp_data = temp_data.set_index(adata.obs.index)
    adata.obs = temp_data
    adata.obs['clinical_AD_Label'] = adata.obs.apply(return_clinical_label,axis=1)
    adata.obs['Pathological_AD_Label'] = adata.obs.apply(return_pthological_label,axis=1)
    adata.obs['Clinical_Pathological_AD_Label'] = adata.obs.apply(return_clinical_pthological_label,axis=1)
    adata.obs.drop('obsnames', axis=1, inplace=True)
    
    for  j in labels:
        print(j)
        subset_adata = adata[adata.obs[j].isin([0.0,1.0]),:]
        Neg = subset_adata.obs[subset_adata.obs[j] ==0.0]
        Pos = subset_adata.obs[subset_adata.obs[j] ==1.0]
        max_sample = min(len(Neg),len(Pos))
        Neg = subset_adata.obs[subset_adata.obs[j] ==0.0].iloc[0:max_sample,:]
        Pos = subset_adata.obs[subset_adata.obs[j] ==1.0].iloc[0:max_sample,:]
        data = pd.concat([Neg,Pos])
        subset_adata = subset_adata[data.index,:]
        count_matrix_df = pd.DataFrame(
                    subset_adata.X.toarray(),  # Convert sparse matrix to dense array if needed
                    index=subset_adata.obs.index,  # Cell IDs as row indices
                    columns=subset_adata.var.index  # Gene names as column indices
                )
    
        
        for l,k in enumerate(genes_set):
            #print(k)
            keep_genes = list(set(k).intersection(set(count_matrix_df.columns)))
            count_matrix_df = count_matrix_df[keep_genes]
            label_df = pd.DataFrame(subset_adata.obs[j])
            name = genes_set_name[l]


             # ML part
            X = torch.tensor(count_matrix_df.values, dtype=torch.float32)
            y = torch.tensor(label_df.values, dtype=torch.long)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
            X_train, X_test, y_train, y_test = X_train.to(device) , X_test.to(device)  , y_train.to(device) , y_test.to(device)

            dataset_train = TensorDataset(X_train, y_train)
            dataset_test = TensorDataset(X_test, y_test)
            input_size = X.shape[1]
            hidden_size =      X.shape[1]//2
            output_size = 1  # Binary classification
            model = NeuralNet(input_size, hidden_size, output_size).to(device) 
            train_loader = DataLoader(dataset_train, batch_size=2048, shuffle=True)
            test_loader = DataLoader(dataset_test, batch_size=2048, shuffle=False)
            criterion = nn.BCEWithLogitsLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.0001)
            best_test_accuracy = train_model(model, train_loader,test_loader, criterion, optimizer, epochs=1000)

            accuracy_dct[f'{i.split('.')[0]}.{name}.{j}'] = best_test_accuracy
            print(accuracy_dct)
             

Per.h5ad
clinical_AD_Label


KeyboardInterrupt: 

In [ ]:
1000//3

In [ ]:
li = []
for i in x:
    if i == True:
        li.append(1)
    else:
        li.append(0)

In [261]:
s = 0
for i,j in enumerate(li):
    if y[i] == j:
        s +=1

In [262]:
s/len(y)

0.53125

In [266]:
s

17

In [21]:
accuracy_dct

{'Per.all_genes.clinical_AD_Label': 86.58536585365853,
 'Per.m6a_genes.clinical_AD_Label': 54.8780487804878,
 'Per.mito_genes.clinical_AD_Label': 50.0,
 'Per.mitophagy_genes.clinical_AD_Label': 50.0,
 'Per.AD_genes.clinical_AD_Label': 50.0,
 'Per.lysosome_genes.clinical_AD_Label': 50.0,
 'Per.all_genes.Pathological_AD_Label': 78.46153846153847,
 'Per.m6a_genes.Pathological_AD_Label': 58.46153846153846,
 'Per.mito_genes.Pathological_AD_Label': 49.23076923076923,
 'Per.mitophagy_genes.Pathological_AD_Label': 49.23076923076923,
 'Per.AD_genes.Pathological_AD_Label': 49.23076923076923,
 'Per.lysosome_genes.Pathological_AD_Label': 49.23076923076923,
 'Opc.all_genes.clinical_AD_Label': 90.56344480073294,
 'Opc.m6a_genes.clinical_AD_Label': 58.77233165368759,
 'Opc.mito_genes.clinical_AD_Label': 50.0229042601924,
 'Opc.mitophagy_genes.clinical_AD_Label': 50.0229042601924,
 'Opc.AD_genes.clinical_AD_Label': 50.0229042601924,
 'Opc.lysosome_genes.clinical_AD_Label': 50.0229042601924,
 'Opc.all_

In [22]:
import pickle

with open('EC.pkl', 'wb') as file:
    pickle.dump(accuracy_dct, file)

print("Dictionary saved as pickle file.")

Dictionary saved as pickle file.


In [13]:
import numpy as np
keep_cell_type = [
 'T.h5ad',
 'Exc.h5ad',
 'Epd.h5ad']

def return_ensemble_id(x,df_ensemble):
    try:
        return df_ensemble[df_ensemble['1']==x]['0'].values[0]
    except: 
        return np.nan

def return_clinical_label(x):
    if x['clinical_diagnosis'] == 'AD':
        return 1.0
    elif x['clinical_diagnosis'] == 'NCI':
        return 0.0
    else:
        return 2.0


def return_pthological_label(x):
    if x['pathological_diagnosis'] == 'AD':
        return 1.0
    elif x['pathological_diagnosis'] == 'No AD':
        return 0.0
    else:
        return 2.0

def return_clinical_pthological_label(x):
    if x['clinical_pathological_AD'] == 'AD_with_Plaques':
        return 1.0
    elif x['clinical_pathological_AD'] == 'NCI_with_No_Plaques':
        return 0.0
    else:
        return 2.0

# Evaluation


clinical_data = pd.read_csv('../clinical_single_cell.csv')

df_ensemble = pd.read_csv('../df_ensemble.csv')

cell_types = []
for i in os.listdir(EC_path):
    if '.h5ad' in i:
        cell_types.append(i)

gene_excel = pd.ExcelFile('../202141022_Genelist_m6aproject_BED.xlsx')
m6a = pd.read_excel(gene_excel, 'm6A')
m6a = m6a.iloc[2:,]
m6a_genes = m6a['m6A related genes/proteins'].tolist()

mito = pd.read_excel(gene_excel, 'Mitochondria')
mito  = mito.iloc[2:,]
mito_genes = mito['Mitochondria genes'].tolist()


mitophagy = pd.read_excel(gene_excel, 'Mitophagy')
mitophagy = mitophagy.iloc[2:,]
mitophagy_genes = mitophagy['Autophagy_Mitophagy_lysosome'].tolist()

AD = pd.read_excel(gene_excel, 'AD')
AD   = AD.iloc[2:,]
AD_genes = AD["Alzheimer's disease - risk factors"].tolist()


mito = pd.read_excel(gene_excel, 'Mitochondria')
mito  = mito.iloc[2:,]
mito_genes = mito['Mitochondria genes'].tolist()
mito_genes[0:5]

lysosome = pd.read_excel(gene_excel, 'Lysosome')
lysosome  = lysosome.iloc[2:,]
lysosome_genes = lysosome['Lysosomes'].tolist()

genes_set  =  [all_genes,m6a_genes,mito_genes,mitophagy_genes,AD_genes,mito_genes,lysosome_genes]
genes_set_name  =  ['all_genes','m6a_genes','mito_genes','mitophagy_genes','AD_genes','mito_genes','lysosome_genes']


        
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            outputs = model(inputs).squeeze(1)
            preds = torch.sigmoid(outputs) > 0.5  # Convert logits to binary predictions
            correct += (preds == labels.squeeze(1)).sum().item()
            total += labels.size(0)
            Accuracy = 100 * correct / total
    return Accuracy

def train_model(model, train_loader,test_loader, criterion, optimizer, epochs=10):
    model.train()
    best_accuracy = 0
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(inputs).squeeze(1)  # Remove extra dimension
            loss = criterion(outputs, labels.squeeze(1).float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        test_accuracy  = evaluate_model(model, test_loader)
        train_accuracy = evaluate_model(model, train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}, TrainAccuracy: {train_accuracy:.2f}, Test_Accuracy: {test_accuracy:.2f}")
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
    
    return best_accuracy




class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

accuracy_dct = dict()

for i in cell_types:
    if i in keep_cell_type:
        print(i)
        adata = sc.read_h5ad(f'{EC_path}{i}')
        temp_data = pd.merge(adata.obs.reset_index(), clinical_data, on =  'projid', how='inner').drop_duplicates()
        temp_data = temp_data.set_index(adata.obs.index)
        adata.obs = temp_data
        adata.obs['clinical_AD_Label'] = adata.obs.apply(return_clinical_label,axis=1)
        adata.obs['Pathological_AD_Label'] = adata.obs.apply(return_pthological_label,axis=1)
        adata.obs['Clinical_Pathological_AD_Label'] = adata.obs.apply(return_clinical_pthological_label,axis=1)
        adata.obs.drop('obsnames', axis=1, inplace=True)
        
        for  j in labels:
            print(j)
            subset_adata = adata[adata.obs[j].isin([0.0,1.0]),:]
            Neg = subset_adata.obs[subset_adata.obs[j] ==0.0]
            Pos = subset_adata.obs[subset_adata.obs[j] ==1.0]
            max_sample = min(len(Neg),len(Pos))
            Neg = subset_adata.obs[subset_adata.obs[j] ==0.0].iloc[0:max_sample,:]
            Pos = subset_adata.obs[subset_adata.obs[j] ==1.0].iloc[0:max_sample,:]
            data = pd.concat([Neg,Pos])
            subset_adata = subset_adata[data.index,:]
            count_matrix_df = pd.DataFrame(
                        subset_adata.X.toarray(),  # Convert sparse matrix to dense array if needed
                        index=subset_adata.obs.index,  # Cell IDs as row indices
                        columns=subset_adata.var.index  # Gene names as column indices
                    )
        
            
            for l,k in enumerate(genes_set):
                #print(k)
                keep_genes = list(set(k).intersection(set(count_matrix_df.columns)))
                count_matrix_df = count_matrix_df[keep_genes]
                label_df = pd.DataFrame(subset_adata.obs[j])
                name = genes_set_name[l]
    
    
                 # ML part
                X = torch.tensor(count_matrix_df.values, dtype=torch.float32)
                y = torch.tensor(label_df.values, dtype=torch.long)
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
                X_train, X_test, y_train, y_test = X_train.to(device) , X_test.to(device)  , y_train.to(device) , y_test.to(device)
    
                dataset_train = TensorDataset(X_train, y_train)
                dataset_test = TensorDataset(X_test, y_test)
                input_size = X.shape[1]
                hidden_size =      X.shape[1]//2
                output_size = 1  # Binary classification
                model = NeuralNet(input_size, hidden_size, output_size).to(device) 
                train_loader = DataLoader(dataset_train, batch_size=2048, shuffle=True)
                test_loader = DataLoader(dataset_test, batch_size=2048, shuffle=False)
                criterion = nn.BCEWithLogitsLoss()
                optimizer = optim.Adam(model.parameters(), lr=0.0001)
                best_test_accuracy = train_model(model, train_loader,test_loader, criterion, optimizer, epochs=200)
    
                accuracy_dct[f'{i.split('.')[0]}.{name}.{j}'] = best_test_accuracy
                print(accuracy_dct)
             

T.h5ad
clinical_AD_Label
Epoch 1/200, Loss: 0.6947, TrainAccuracy: 64.18, Test_Accuracy: 58.21
Epoch 2/200, Loss: 0.5852, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 3/200, Loss: 1.0781, TrainAccuracy: 75.75, Test_Accuracy: 60.45
Epoch 4/200, Loss: 0.4876, TrainAccuracy: 64.74, Test_Accuracy: 56.72
Epoch 5/200, Loss: 0.5307, TrainAccuracy: 58.77, Test_Accuracy: 53.73
Epoch 6/200, Loss: 0.5714, TrainAccuracy: 71.46, Test_Accuracy: 56.72
Epoch 7/200, Loss: 0.4594, TrainAccuracy: 96.27, Test_Accuracy: 64.93
Epoch 8/200, Loss: 0.3797, TrainAccuracy: 98.88, Test_Accuracy: 70.90
Epoch 9/200, Loss: 0.3484, TrainAccuracy: 96.27, Test_Accuracy: 70.15
Epoch 10/200, Loss: 0.3248, TrainAccuracy: 97.01, Test_Accuracy: 70.90
Epoch 11/200, Loss: 0.2678, TrainAccuracy: 99.44, Test_Accuracy: 73.13
Epoch 12/200, Loss: 0.1847, TrainAccuracy: 99.44, Test_Accuracy: 74.63
Epoch 13/200, Loss: 0.1255, TrainAccuracy: 99.07, Test_Accuracy: 68.66
Epoch 14/200, Loss: 0.1013, TrainAccuracy: 99.44, Test_Accura

/home/anwer/.local/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Epoch 2/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 3/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 4/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 5/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 6/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 7/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 8/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 9/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 10/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 11/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 12/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 13/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 14/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 15/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 16/200, 

/home/anwer/.local/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Epoch 23/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 24/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 25/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 26/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 27/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 28/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 29/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 30/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 31/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 32/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 33/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 34/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 35/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 36/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 

...

/home/anwer/.local/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Epoch 1/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 2/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 3/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 4/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 5/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 6/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 7/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 8/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 9/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 10/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 11/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 12/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 13/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 14/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 15/200, L

/home/anwer/.local/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Epoch 1/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 2/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 3/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 4/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 5/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 6/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 7/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 8/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 9/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 10/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 11/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 12/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 13/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 14/200, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 15/200, L

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [14]:
accuracy_dct

{'T.all_genes.clinical_AD_Label': 78.35820895522389,
 'T.m6a_genes.clinical_AD_Label': 52.985074626865675,
 'T.mito_genes.clinical_AD_Label': 50.0,
 'T.mitophagy_genes.clinical_AD_Label': 50.0,
 'T.AD_genes.clinical_AD_Label': 50.0,
 'T.lysosome_genes.clinical_AD_Label': 50.0,
 'T.all_genes.Pathological_AD_Label': 80.9090909090909,
 'T.m6a_genes.Pathological_AD_Label': 56.36363636363637,
 'T.mito_genes.Pathological_AD_Label': 50.0,
 'T.mitophagy_genes.Pathological_AD_Label': 50.0,
 'T.AD_genes.Pathological_AD_Label': 50.0,
 'T.lysosome_genes.Pathological_AD_Label': 50.0,
 'Exc.all_genes.clinical_AD_Label': 97.48436414176511,
 'Exc.m6a_genes.clinical_AD_Label': 59.97220291869354,
 'Exc.mito_genes.clinical_AD_Label': 50.006949270326615,
 'Exc.mitophagy_genes.clinical_AD_Label': 50.006949270326615,
 'Exc.AD_genes.clinical_AD_Label': 50.006949270326615,
 'Exc.lysosome_genes.clinical_AD_Label': 50.006949270326615,
 'Exc.all_genes.Pathological_AD_Label': 98.29199812821713,
 'Exc.m6a_genes.Pa

In [16]:
import pickle
with open("EC.pkl", "rb") as file:
    data = pickle.load(file)

In [18]:
data.update(accuracy_dct)

In [20]:
import pickle

with open('EC.pkl', 'wb') as file:
    pickle.dump(data, file)

print("Dictionary saved as pickle file.")

Dictionary saved as pickle file.
